In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.preprocessing import OneHotEncoder
import joblib
from model import RegNet

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
input_size = 847
hidden_sizes = [512, 128]
output_size = 1

In [ ]:
model = RegNet(input_size,hidden_sizes,output_size)
model.load_state_dict(torch.load(f'new_new_model_epoch_1.pth'))

In [ ]:
torch_params = {}
for i, layer in enumerate(model.network):
    if isinstance(layer, nn.Linear):
        torch_params[f'coefs_{i//2}'] = layer.weight.detach().numpy().T  # 转置匹配 sklearn 格式
        torch_params[f'intercepts_{i//2}'] = layer.bias.detach().numpy()

In [ ]:
# 定义 sklearn MLPRegressor
mlp_model = MLPRegressor(hidden_layer_sizes=(512, 128),
                         activation='relu',
                         solver='adam',
                         max_iter=1)  # max_iter=1 是为了初始化模型，不进行真正的训练

# 训练一次来初始化模型参数

X_dummy = np.random.rand(10, 847)
y_dummy = np.random.rand(10)
mlp_model.fit(X_dummy, y_dummy)

In [ ]:
for i in range(len(hidden_sizes) + 1):  # 遍历所有层
    mlp_model.coefs_[i] = torch_params[f'coefs_{i}']
    mlp_model.intercepts_[i] = torch_params[f'intercepts_{i}']

In [ ]:
joblib.dump(mlp_model, 'new_new_model_1.pkl')